In [1]:

import pandas
import requests
import spotipy
import string
import re
import csv

from spotipy.oauth2 import SpotifyClientCredentials

with open('vars.env', 'r') as f:
    my_id = f.readline().rstrip('\n')
    my_secret = f.readline().rstrip('\n')
    f.readline()
    f.readline()
    playlistURI = f.readline().rstrip('\n')
    owner = f.readline().rstrip('\n')

client_credentials_manager = SpotifyClientCredentials(client_id=my_id, client_secret=my_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlistID = playlistURI[-22:]

results = sp.user_playlist(owner, playlist_id=playlistID, fields='tracks, next')['tracks']

songs =  results['items']
while results['next']:
    results = sp.next(results)
    songs.extend(results['items'])

idSet = {track['added_by']['id'] for track in songs}    
us = {(id, sp.user(id)['display_name']) for id in idSet}
us = dict(us)
us['rmb101'] = 'rmb101'

# Make the dataframe by pulling out the song name, artist and person who added it
table = [(track['track']['name'], track['track']['artists'][0]['name'], track['track']['album']['name'], us[track['added_by']['id']]) 
         for track in songs]
df = pandas.DataFrame(table, columns = ['Song', 'Artist', 'Album', 'Added by'])

In [2]:
cols = ['colour', 'color','red', 'orange', 'yellow', 'green', 'blue', 'purple', 'pink', 'brown', 'gray', 'black', 'white', 'gold', 'lilac', 'xanadu', 'mikado', 'midnight', 'octarine', 'rust', 'mustard']

more = 'White silver grey black navy blue cerulean sky blue turquoise blue-green azure teal cyan green lime chartreuse olive yellow gold amber orange brown orange-red red maroon rose red-violet pink magenta purple blue-violet indigo violet peach apricot ochre plum'.split()

cols.extend(more)

colCount = {}

text = df['Song'].str.lower().to_string() + df['Album'].str.lower().to_string() 

for hue in cols: 
    count = text.count(hue)
    if count > 0:
        colCount[hue] = count
        
colCount['colour or color'] = colCount['color'] + colCount['colour']
del colCount['color']
del colCount['colour']

In [3]:
results = [(colour, count) for colour, count in colCount.items()]
results.sort(key=lambda pair: -pair[1])

In [4]:
with open('colours.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(results)
